Great question — and you're right to think deeply about this. It **depends on your objective**. Here's a breakdown to help you decide between **binary classification** (real vs fake) and **multiclass classification** (real, Deepfakes, Face2Face, FaceSwap, NeuralTextures, etc.).

---

### ✅ **Binary Classification (Real vs Fake)**

**Pros:**

* **Simpler model** and easier to train.
* **More data per class** (especially if dataset is imbalanced across manipulation methods).
* Works well if your **goal is just detection** of whether a video is fake.

**Cons:**

* You **lose information** about *how* it was faked.
* May be **less interpretable** in forensic or research contexts.

**Recommended if:**

* You’re focused on building a **robust detector**, regardless of the manipulation method.
* You have limited or unbalanced data per fake class.

---

### ✅ **Multiclass Classification (Real, Deepfakes, FaceSwap, etc.)**

**Pros:**

* Provides **more granular insights** about the type of manipulation.
* Useful in forensic, academic, or investigative settings.
* A softmax output gives **probability distribution**, which can help flag ambiguous or unknown manipulations (e.g., 0.3 Deepfake, 0.3 FaceSwap, 0.4 other).

**Cons:**

* Needs **balanced and cleanly labeled data** across all manipulation types.
* If a fake video was created using a **method not in your training set**, the model may misclassify it with high confidence.
* Models may **confuse manipulation types** if they have similar artifacts.

**Recommended if:**

* You’re working in a **forensic context** or need to **analyze manipulation methods**.
* You have a **well-balanced dataset** across fake types (or can artificially balance it).
* You plan to extend the system to **open-set recognition** or **anomaly detection** later.

---

### 🚧 Hybrid Option: Binary + Softmax Confidence

You could:

1. Train a **binary classifier** (real vs fake).
2. Train a **secondary classifier** to **predict manipulation type**, *only on the fakes*.
3. Optionally, expose softmax probabilities as part of **confidence scoring** for human-in-the-loop review.

This allows:

* High accuracy on real vs fake.
* Flexibility in analysis (showing which type it's closest to).
* Safer handling of *unknown fakes*.

---

### 🔍 Real-World Considerations

* Many real-world videos may be manipulated in **unknown or novel ways**.
* A softmax model trained on 4 fake classes might fail when presented with a **fifth** unseen type.
* Consider using **OOD (out-of-distribution) detection** techniques or **embedding-based methods** later if you want to generalize more robustly.

---

### 🧠 Summary

| Objective                              | Recommendation                            |
| -------------------------------------- | ----------------------------------------- |
| Just detect fake videos                | **Binary classification** ✅               |
| Analyze or visualize manipulation type | **Multiclass classification** or hybrid ✅ |
| Expect novel/unseen manipulations      | Binary + OOD detection or hybrid ✅        |
| Dataset is small or unbalanced         | Start with binary ✅                       |

Would you like help designing a hybrid setup or deciding on loss functions or model architectures for either approach?


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import urllib.request, json

url = "http://kaldir.vc.in.tum.de/faceforensics/v3/misc/deepfake_detection_filenames.json"
with urllib.request.urlopen(url) as f:
    detection_files = json.loads(f.read().decode())

print(detection_files.keys())


dict_keys(['actors', 'DeepFakesDetection'])


In [3]:
####### REMEMBER TO DELETE THE OLD VIDEOS BEFORE LOADING NEW ONES ###########
####### REMEMBER TO DELETE THE OLD VIDEOS BEFORE LOADING NEW ONES ###########
####### REMEMBER TO DELETE THE OLD VIDEOS BEFORE LOADING NEW ONES ###########
####### REMEMBER TO DELETE THE OLD VIDEOS BEFORE LOADING NEW ONES ###########
# !python /content/drive/MyDrive/Colab_Notebooks/GAIDI/Deepfake/faceforensic_downloader_colab.py
####### REMEMBER TO DELETE THE OLD VIDEOS BEFORE LOADING NEW ONES ###########
####### REMEMBER TO DELETE THE OLD VIDEOS BEFORE LOADING NEW ONES ###########
####### REMEMBER TO DELETE THE OLD VIDEOS BEFORE LOADING NEW ONES ###########

Already downloaded: 1252

🔻 Downloading 240 new videos...
100% 240/240 [09:05<00:00,  2.27s/it]

✅ Done: 240 new videos downloaded.


#Module 1: Frame- based Detection
##Pipeline Overview
Extract Frames from video (e.g., every 1-2 frames per second).

Preprocess: Resize, normalize, optionally crop faces.

Model: CNN or ViT trained on real vs fake images.

Aggregate predictions across frames → one score per video.
##Tools & Libraries
OpenCV or ffmpeg-python for frame extraction.

PyTorch / TensorFlow for model training.

Pretrained models: ResNet50, EfficientNet, or ViT.

In [4]:
import os
# !pip install opencv-python
import cv2
print(cv2.__version__)

4.12.0


# Frames' extraction

## fix number of frames per videos
Balanced dataset

In [5]:
########### FOR FIX NUMBER OF FRAME PER VIDEOS #########
# import cv2
# import os
# import numpy as np

# deepf_path = '/content/drive/MyDrive/Colab_Notebooks/GAIDI/Deepfake/manipulated/DeepFakeDetection'
# output_path = '/content/drive/MyDrive/Colab_Notebooks/GAIDI/Deepfake/data/fake'
# frames_per_video = 10

# os.makedirs(output_path, exist_ok=True)

# for name in os.listdir(deepf_path):
#     if not name.endswith('.mp4'):
#         print(f"Skipping {name}, not a video file")
#         continue

#     video_file = os.path.join(deepf_path, name)
#     cap = cv2.VideoCapture(video_file)

#     if not cap.isOpened():
#         print(f'❌ Error opening {name}')
#         continue

#     total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
#     print(f'🎞️ {name}: total frames = {total_frames}')

#     if total_frames < frames_per_video:
#         print(f"⚠️  {name} has fewer frames than requested ({frames_per_video})")
#         frame_indices = list(range(total_frames))  # take all
#     else:
#         frame_indices = np.linspace(0, total_frames - 1, frames_per_video, dtype=int)

#     saved_count = 0
#     for i, target_idx in enumerate(frame_indices):
#         cap.set(cv2.CAP_PROP_POS_FRAMES, target_idx)
#         ret, frame = cap.read()
#         if ret:
#             out_path = os.path.join(output_path, f"{name}_{i:04d}.jpg")
#             cv2.imwrite(out_path, frame)
#             saved_count += 1

#     cap.release()
#     print(f'✅ {saved_count} frames saved from {name} to {output_path}')


## ORIGINAL VIDEOS FRAME EXTRACTION

In [ ]:
OG_path = '/content/drive/MyDrive/Colab_Notebooks/GAIDI/Deepfake/originals'
output_path = '/content/drive/MyDrive/Colab_Notebooks/GAIDI/Deepfake/data/real'
wanted_fps = 3

### for deployment later 
### video_extensions = (".mp4", ".webm", ".mkv", ".avi")
os.makedirs(output_path, exist_ok=True)

for name in os.listdir(OG_path): #iterate through the files in OG_path
    if name.endswith('.mp4'): ### if name.lower().endswith(video_extensions):
        OG_path_mp4 = os.path.join(OG_path, name)
        cap = cv2.VideoCapture(OG_path_mp4)

        if not cap.isOpened():
            print("Error: Could not open video.")
            exit()

        video_fps= cap.get(cv2.CAP_PROP_FPS)
        frame_interval = int(video_fps / wanted_fps)

        print(f'Video FPS: {video_fps}, capturing a frame every {frame_interval} frames')

        frame_count = 0
        saved_count = 0

        while True:
            ret, frame = cap.read()

            if not ret:
                break

            if frame_count % frame_interval == 0:
                output_filepath  = os.path.join(output_path, f'{name}_{saved_count:04d}.jpg')
                cv2.imwrite(output_filepath, frame)
                saved_count += 1
            frame_count += 1
        cap.release()
        print(f'Finish, {saved_count} frames saved to {output_path}')
    else:
        print(f'Skipping {name}, not a video file')

Video FPS: 25.0, capturing a frame every 8 frames
Finish, 46 frames saved to /content/drive/MyDrive/Colab_Notebooks/GAIDI/Deepfake/data/real
Video FPS: 25.0, capturing a frame every 8 frames
Finish, 43 frames saved to /content/drive/MyDrive/Colab_Notebooks/GAIDI/Deepfake/data/real
Video FPS: 30.0, capturing a frame every 10 frames
Finish, 35 frames saved to /content/drive/MyDrive/Colab_Notebooks/GAIDI/Deepfake/data/real
Video FPS: 30.0, capturing a frame every 10 frames
Finish, 33 frames saved to /content/drive/MyDrive/Colab_Notebooks/GAIDI/Deepfake/data/real
Video FPS: 25.0, capturing a frame every 8 frames
Finish, 74 frames saved to /content/drive/MyDrive/Colab_Notebooks/GAIDI/Deepfake/data/real
Video FPS: 25.0, capturing a frame every 8 frames
Finish, 52 frames saved to /content/drive/MyDrive/Colab_Notebooks/GAIDI/Deepfake/data/real
Video FPS: 25.0, capturing a frame every 8 frames
Finish, 42 frames saved to /content/drive/MyDrive/Colab_Notebooks/GAIDI/Deepfake/data/real
Video FPS: 

## Manipulated sequences Frame Extraction

#### DeepFakesDetection

In [7]:
deepf_path = '/content/drive/MyDrive/Colab_Notebooks/GAIDI/Deepfake/manipulated/DeepFakeDetection'
output_path = '/content/drive/MyDrive/Colab_Notebooks/GAIDI/Deepfake/data/fake'
wanted_fps = 3

os.makedirs(output_path, exist_ok=True)


for name in os.listdir(deepf_path):
    if name.endswith('.mp4'):
        deepf_path_mp4 = os.path.join(deepf_path,name)
        cap = cv2.VideoCapture(deepf_path_mp4)

        if not cap.isOpened():
            print('Error: cannot open videos.mp4')
            exit()

        videos_fps = cap.get(cv2.CAP_PROP_FPS)
        frame_interval = int(videos_fps/wanted_fps)
        print(f'Video FPS: {videos_fps}, capturing a frame every {frame_interval} frames')

        frame_count = 0
        saved_count = 0

        while True:
            ret, frame = cap.read()

            if not ret:
                break

            if frame_count % frame_interval == 0:
                output_filepath  = os.path.join(output_path,f'{name}_{saved_count:04d}.jpg')
                cv2.imwrite(output_filepath , frame)
                saved_count += 1

            frame_count += 1
        cap.release()
        print(f'Finish, {saved_count} frames saved to {output_path}')
    else:
        print(f'Skipping {name}, not a video file')

Video FPS: 24.0, capturing a frame every 8 frames
Finish, 130 frames saved to /content/drive/MyDrive/Colab_Notebooks/GAIDI/Deepfake/data/fake
Video FPS: 24.0, capturing a frame every 8 frames
Finish, 108 frames saved to /content/drive/MyDrive/Colab_Notebooks/GAIDI/Deepfake/data/fake
Video FPS: 24.0, capturing a frame every 8 frames
Finish, 99 frames saved to /content/drive/MyDrive/Colab_Notebooks/GAIDI/Deepfake/data/fake
Video FPS: 24.0, capturing a frame every 8 frames
Finish, 60 frames saved to /content/drive/MyDrive/Colab_Notebooks/GAIDI/Deepfake/data/fake
Video FPS: 24.0, capturing a frame every 8 frames
Finish, 122 frames saved to /content/drive/MyDrive/Colab_Notebooks/GAIDI/Deepfake/data/fake
Video FPS: 24.0, capturing a frame every 8 frames
Finish, 118 frames saved to /content/drive/MyDrive/Colab_Notebooks/GAIDI/Deepfake/data/fake
Video FPS: 24.0, capturing a frame every 8 frames
Finish, 64 frames saved to /content/drive/MyDrive/Colab_Notebooks/GAIDI/Deepfake/data/fake
Video FPS

#### Deepfake

In [8]:
df_path = '/content/drive/MyDrive/Colab_Notebooks/GAIDI/Deepfake/manipulated/Deepfakes'
output_path = '/content/drive/MyDrive/Colab_Notebooks/GAIDI/Deepfake/data/fake'
desired_fps = 3

os.makedirs(output_path, exist_ok=True)

for name in os.listdir(df_path):
    if name.endswith('.mp4'):
        fd_path_mp4 = os.path.join(df_path, name)

        cap = cv2.VideoCapture(fd_path_mp4)

        if not cap.isOpened():
            print('Error')
            exit()

        video_fps = cap.get(cv2.CAP_PROP_FPS)
        frame_interval = int(video_fps/frame_interval)
        print(f'Video FPS:{video_fps}, capturing a frame every {frame_interval} frames.')

        frame_count = 0
        saved_count = 0

        while True:
            ret, frame = cap.read()

            if not ret:
                break

            if frame_count % frame_interval == 0:
                output_filepath  = os.path.join(output_path,f'{name}_{saved_count:04d}.jpg')
                cv2.imwrite(output_filepath , frame)
                saved_count += 1
            frame_count += 1

        cap.release()

        print(f'Done, saved {saved_count} frames in {output_path}')
    else:
        print(f'Skipping {name}, not a video file')

Video FPS:25.0, capturing a frame every 3 frames.
Done, saved 150 frames in /content/drive/MyDrive/Colab_Notebooks/GAIDI/Deepfake/data/fake
Video FPS:24.0, capturing a frame every 8 frames.
Done, saved 41 frames in /content/drive/MyDrive/Colab_Notebooks/GAIDI/Deepfake/data/fake
Video FPS:25.0, capturing a frame every 3 frames.
Done, saved 336 frames in /content/drive/MyDrive/Colab_Notebooks/GAIDI/Deepfake/data/fake
Video FPS:25.0, capturing a frame every 8 frames.
Done, saved 51 frames in /content/drive/MyDrive/Colab_Notebooks/GAIDI/Deepfake/data/fake
Video FPS:25.0, capturing a frame every 3 frames.
Done, saved 104 frames in /content/drive/MyDrive/Colab_Notebooks/GAIDI/Deepfake/data/fake
Video FPS:25.0, capturing a frame every 8 frames.
Done, saved 61 frames in /content/drive/MyDrive/Colab_Notebooks/GAIDI/Deepfake/data/fake
Video FPS:25.0, capturing a frame every 3 frames.
Done, saved 104 frames in /content/drive/MyDrive/Colab_Notebooks/GAIDI/Deepfake/data/fake
Video FPS:30.0, capturi

#### Face2Face

In [9]:
f2f_path = '/content/drive/MyDrive/Colab_Notebooks/GAIDI/Deepfake/manipulated/Face2Face'
output_path = '/content/drive/MyDrive/Colab_Notebooks/GAIDI/Deepfake/data/fake'
desired_fps = 3

os.makedirs(output_path, exist_ok=True)

for name in os.listdir(f2f_path):
    if name.endswith('.mp4'):
        f2f_path_mp4 = os.path.join(f2f_path, name)

        cap=cv2.VideoCapture(f2f_path_mp4)

        if not cap.isOpened():
            print('Error: Videos cannot be opened')
            exit()

        video_fps = cap.get(cv2.CAP_PROP_FPS)

        frame_interval = int(video_fps/desired_fps)
        print(f'Videos FPS:{video_fps}, capturing a frame every {frame_interval} frames')

        frame_count =0
        saved_count= 0

        while True:
            ret, frame = cap.read()

            if not ret:
                break

            if frame_count % frame_interval == 0:
                output_filepath  = os.path.join(output_path, f'{name}_{saved_count:04d}.jpg')
                cv2.imwrite(output_filepath , frame)
                saved_count += 1
            frame_count += 1

        cap.release()
        print(f'Finish, {saved_count} frames saved to {output_path}')
    else:
        print(f'Skipping {name}, not a video file')

Videos FPS:24.0, capturing a frame every 8 frames
Finish, 126 frames saved to /content/drive/MyDrive/Colab_Notebooks/GAIDI/Deepfake/data/fake
Videos FPS:25.0, capturing a frame every 8 frames
Finish, 61 frames saved to /content/drive/MyDrive/Colab_Notebooks/GAIDI/Deepfake/data/fake
Videos FPS:30.0, capturing a frame every 10 frames
Finish, 143 frames saved to /content/drive/MyDrive/Colab_Notebooks/GAIDI/Deepfake/data/fake
Videos FPS:30.0, capturing a frame every 10 frames
Finish, 31 frames saved to /content/drive/MyDrive/Colab_Notebooks/GAIDI/Deepfake/data/fake
Videos FPS:30.0, capturing a frame every 10 frames
Finish, 30 frames saved to /content/drive/MyDrive/Colab_Notebooks/GAIDI/Deepfake/data/fake
Videos FPS:25.0, capturing a frame every 8 frames
Finish, 111 frames saved to /content/drive/MyDrive/Colab_Notebooks/GAIDI/Deepfake/data/fake
Videos FPS:25.0, capturing a frame every 8 frames
Finish, 151 frames saved to /content/drive/MyDrive/Colab_Notebooks/GAIDI/Deepfake/data/fake
Videos

#### FaceShifter

In [10]:
fshift_path = '/content/drive/MyDrive/Colab_Notebooks/GAIDI/Deepfake/manipulated/FaceShifter'
output_path = '/content/drive/MyDrive/Colab_Notebooks/GAIDI/Deepfake/data/fake'
desired_fps = 3

os.makedirs(output_path,exist_ok=True)

for name in os.listdir(fshift_path):
    if name.endswith('.mp4'):
        fshift_path_mp4 = os.path.join(fshift_path,name)

        cap = cv2.VideoCapture(fshift_path_mp4)

        if not cap.isOpened():
            print("Error: Cannot read the video")
            exit()

        video_fps = cap.get(cv2.CAP_PROP_FPS)
        frame_interval = int(video_fps/desired_fps)
        print(f'Video FPS:{video_fps}, capturing a frame every {frame_interval} frames.')

        frame_count = 0
        saved_count = 0

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            if frame_count % frame_interval == 0:
                output_filepath  = os.path.join(output_path,f'{name}_{saved_count:04d}.jpg')
                cv2.imwrite(output_filepath, frame)
                saved_count += 1
            frame_count += 1

        cap.release()

        print(f'Finish, {saved_count} frames saved to {output_path}')
    else:
        print(f'Skipping {name}, not a video file')

Video FPS:25.0, capturing a frame every 8 frames.
Finish, 53 frames saved to /content/drive/MyDrive/Colab_Notebooks/GAIDI/Deepfake/data/fake
Video FPS:25.0, capturing a frame every 8 frames.
Finish, 42 frames saved to /content/drive/MyDrive/Colab_Notebooks/GAIDI/Deepfake/data/fake
Video FPS:25.0, capturing a frame every 8 frames.
Finish, 87 frames saved to /content/drive/MyDrive/Colab_Notebooks/GAIDI/Deepfake/data/fake
Video FPS:30.0, capturing a frame every 10 frames.
Finish, 30 frames saved to /content/drive/MyDrive/Colab_Notebooks/GAIDI/Deepfake/data/fake
Video FPS:30.0, capturing a frame every 10 frames.
Finish, 31 frames saved to /content/drive/MyDrive/Colab_Notebooks/GAIDI/Deepfake/data/fake
Video FPS:25.0, capturing a frame every 8 frames.
Finish, 70 frames saved to /content/drive/MyDrive/Colab_Notebooks/GAIDI/Deepfake/data/fake
Video FPS:30.0, capturing a frame every 10 frames.
Finish, 70 frames saved to /content/drive/MyDrive/Colab_Notebooks/GAIDI/Deepfake/data/fake
Video FPS:

#### FaceSwap

In [11]:
fswap_path = '/content/drive/MyDrive/Colab_Notebooks/GAIDI/Deepfake/manipulated/FaceSwap'
output_path = '/content/drive/MyDrive/Colab_Notebooks/GAIDI/Deepfake/data/fake'
desired_fps = 3

os.makedirs(output_path, exist_ok=True)

for name in os.listdir(fswap_path):
    if name.endswith('.mp4'):
        fswap_path_mp4 = os.path.join(fswap_path, name)

        cap = cv2.VideoCapture(fswap_path_mp4)

        if not cap.isOpened():
            print('Error: Connot open the video')
            exit()

        video_fps = cap.get(cv2.CAP_PROP_FPS)
        frame_interval = int(video_fps / desired_fps)
        print(f'Videos FPS: {video_fps}, capturing a frame every {frame_interval} frames.')

        frame_count = 0
        saved_count = 0

        while True:
            ret, frame = cap.read()

            if not ret:
                break

            if frame_count % frame_interval == 0:
                output_filepath  = os.path.join(output_path,f'{name}_{saved_count:04d}.jpg')
                cv2.imwrite(output_filepath, frame)
                saved_count += 1

            frame_count += 1
        cap.release()

        print(f'Finish, {saved_count} frames saved to {output_path}')
    else:
        print(f'Skipping {name}, not a video file')

Videos FPS: 30.0, capturing a frame every 10 frames.
Finish, 37 frames saved to /content/drive/MyDrive/Colab_Notebooks/GAIDI/Deepfake/data/fake
Videos FPS: 25.0, capturing a frame every 8 frames.
Finish, 49 frames saved to /content/drive/MyDrive/Colab_Notebooks/GAIDI/Deepfake/data/fake
Videos FPS: 30.0, capturing a frame every 10 frames.
Finish, 34 frames saved to /content/drive/MyDrive/Colab_Notebooks/GAIDI/Deepfake/data/fake
Videos FPS: 30.0, capturing a frame every 10 frames.
Finish, 36 frames saved to /content/drive/MyDrive/Colab_Notebooks/GAIDI/Deepfake/data/fake
Videos FPS: 24.0, capturing a frame every 8 frames.
Finish, 41 frames saved to /content/drive/MyDrive/Colab_Notebooks/GAIDI/Deepfake/data/fake
Videos FPS: 25.0, capturing a frame every 8 frames.
Finish, 52 frames saved to /content/drive/MyDrive/Colab_Notebooks/GAIDI/Deepfake/data/fake
Videos FPS: 30.0, capturing a frame every 10 frames.
Finish, 39 frames saved to /content/drive/MyDrive/Colab_Notebooks/GAIDI/Deepfake/data/

#### NeuralTexture

In [12]:
NT_path = '/content/drive/MyDrive/Colab_Notebooks/GAIDI/Deepfake/manipulated/NeuralTextures'
output_path = '/content/drive/MyDrive/Colab_Notebooks/GAIDI/Deepfake/data/fake'
desired_fps = 3

os.makedirs(output_path, exist_ok=True)

for name in os.listdir(NT_path):
    if name.endswith('mp4'):
        NT_path_mp4 = os.path.join(NT_path,name)

        cap = cv2.VideoCapture(NT_path_mp4)

        if not cap.isOpened():
            print("Error: cannot read video")
            exit()

        video_fps = cap.get(cv2.CAP_PROP_FPS)
        frame_interval = int(video_fps / desired_fps)

        print(f'Video FPS: {video_fps}, capturing 1 frame every {frame_interval} frames.')

        frame_count = 0
        saved_count = 0

        while True:
            ret, frame = cap.read()

            if not ret:
                break

            if frame_count % frame_interval ==0:
                output_filepath = os.path.join(output_path,f'{name}_{saved_count:04d}.jpg')
                cv2.imwrite(output_filepath, frame)
                saved_count += 1

            frame_count += 1

        cap.release()

        print(f'Done, {saved_count} saved in {output_path}')
    else:
        print('Skipped, cannot read the video')

Video FPS: 30.0, capturing 1 frame every 10 frames.
Done, 52 saved in /content/drive/MyDrive/Colab_Notebooks/GAIDI/Deepfake/data/fake
Video FPS: 30.0, capturing 1 frame every 10 frames.
Done, 34 saved in /content/drive/MyDrive/Colab_Notebooks/GAIDI/Deepfake/data/fake
Video FPS: 30.0, capturing 1 frame every 10 frames.
Done, 35 saved in /content/drive/MyDrive/Colab_Notebooks/GAIDI/Deepfake/data/fake
Video FPS: 30.0, capturing 1 frame every 10 frames.
Done, 43 saved in /content/drive/MyDrive/Colab_Notebooks/GAIDI/Deepfake/data/fake
Video FPS: 30.0, capturing 1 frame every 10 frames.
Done, 52 saved in /content/drive/MyDrive/Colab_Notebooks/GAIDI/Deepfake/data/fake
Video FPS: 30.0, capturing 1 frame every 10 frames.
Done, 61 saved in /content/drive/MyDrive/Colab_Notebooks/GAIDI/Deepfake/data/fake
Video FPS: 30.0, capturing 1 frame every 10 frames.
Done, 46 saved in /content/drive/MyDrive/Colab_Notebooks/GAIDI/Deepfake/data/fake
Video FPS: 25.0, capturing 1 frame every 8 frames.
Done, 66 sa

In [13]:
# Play an audio beep. Any audio URL will do.
from google.colab import output
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/5/57/BP_Just_D.ogg").play()')